In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class ST:
  def __init__(self, L, f, default=0):
    self.default = default
    self.f = f
    self.len = len(L)
    self.size = size = 1 << (self.len - 1).bit_length()

    self.seg = [default] * (2 * size)
    self.seg[size:size + self.len] = L
    for i in reversed(range(size)):
      self.seg[i] = f(self.seg[i + i], self.seg[i + i + 1])

  def __getitem__(self, i):
    return self.seg[i + self.size]

  def __setitem__(self, i, v):
    i += self.size
    self.seg[i] = v
    i //= 2
    while i:
      self.seg[i] = self.f(self.seg[2 * i], self.seg[2 * i + 1])
      i //= 2

  def query(self, s, e): # [s, e)
    s += self.size
    e += self.size

    l = r = self.default
    while s < e: 
      if s & 1:
        l = self.f(l, self.seg[s])
        s += 1
      if e & 1:
        e -= 1
        r = self.f(self.seg[e], r)
      s //= 2
      e //= 2

    return self.f(l, r)

INF = 10 ** 9 + 7
def sol():
  N = int(input())
  L = [[] for _ in range(N)]
  for _ in range(3) :
    l = list(map(int, input().split()))
    for i, v in enumerate(l) :
      L[v-1].append(i)
  
  L.sort(key=lambda x: x[0])
  st = ST([INF] * N, min, INF)
  ans = 0
  for x, y, z in L :
    if st.query(0, y) > z:
      ans += 1
    st[y] = z

  print(ans)

sol()

### 풀이
- 하나의 criteria로 우선 정렬한다. 풀이에선 1 번째 시험 점수로 정렬했다.
  - 동일한 등수가 없으므로, 순서로 앞선 학생은 적어도 뒤에있는 학생보다 대단하다.
- 이후 2, 3번째 시험을 동시에 잘 본 학생들을 확인하여 다음 기준으로 정렬한다.
  - 특정 학생보다 앞에있는 학생 중 2, 3번째 시험을 동시에 잘본 학생이 없다면, 해당 학생은 굉장한 학생이다.
- 기준이 딱 3개 되는 문제에서 가능한 타입이다.
  - 2번째 기준으로 쿼리 한 뒤, 그 위치에 3번째 시험의 값을 넣는다.
  - 이렇게 되면 1번째 기준은 정렬성, 2번째 기준은 쿼리의 인덱스, 3번째 위치는 값으로 순위가 결정된다.